<a href="https://colab.research.google.com/github/khushboo-garg/SMCR/blob/main/Ques_3_5_Earth_Heat_2N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SMCR HW 2 _ Khushboo Garg

In [ ]:
import numpy as np , matplotlib.pyplot as plt

# Earth Heat 2N

In [ ]:
# ODE4U function
def ode4u(odefun, time, x0, u=None, c=None):
    """
    Solve a system of nonhomogeneous ODEs using the 4th-order Runge-Kutta method.
    (it depends on not just time and state but also external inputs (u) and constanstant (c))

    Parameters:
        odefun : function (t, x, u, c) -> (dxdt, y)
                 returns state derivative and output as arrays
        time   : time values at which the solution is computed.
                 p-dimensional array
        x0     : n-dimensional array, state at time[0].
        u      : (m x p) dimensional array
                 optional input sampled at each time step.
        c      : optional constants passed to odefun.

    Returns:
        time   : ndarray, shape (1, p)
        x_sol  : ndarray, shape (n, p)
        x_drv  : ndarray, shape (n, p)
        y_sol  : ndarray, shape (m, p)
    """

    time   = np.asarray(time)
    x0     = np.asarray(x0).flatten()
    points = len(time) # the total number of time steps

    # create defaults if not provided
    if c is None:
        c = 0
    if u is None:
        u = np.zeros((1, points))
    else:
        u = np.asarray(u)

    # verify inputs recieved are in 2D array shape
    if u.ndim == 1:
        u = u[np.newaxis, :]

    if u.shape[1] < points:
        pad_width = points - u.shape[1]
        u = np.pad(u, ((0, 0), (0, pad_width)), mode='constant')

    # state derivitives and outputs at time[0]
    dxdt0, y0 = odefun(time[0], x0, u[:, 0], c)

    n = x0.size                 # number of states
    m = np.asarray(y0).size     # nuber of outputs

    # allocate memory
    x_sol = np.ones([n, points])*np.nan
    x_drv = np.ones([n, points])*np.nan
    y_sol = np.ones([m, points])*np.nan

    x_sol[:, 0] = x0    # states
    x_drv[:, 0] = dxdt0 # state derivitives
    y_sol[:, 0] = y0    # outputs

    for p in range(points - 1):  # time stepping loop and main integration
        t = time[p]
        dt = time[p + 1] - t
        dt2 = dt / 2.0

        u_mid = (u[:, p] + u[:, p + 1]) / 2.0

        # intermediate dervitives
        dxdt1, _ = odefun(t + dt2, x0 + dxdt0 * dt2, u_mid, c)
        dxdt2, _ = odefun(t + dt2, x0 + dxdt1 * dt2, u_mid, c)
        dxdt3, _ = odefun(t + dt,  x0 + dxdt2 * dt, u[:, p + 1], c)

        # state update using the intermediate derivities
        x0 = x0 + ( dxdt0 + 2 * (dxdt1 + dxdt2) + dxdt3 ) * dt / 6.0

        # state derivitives and outputs at start of time (p+1)
        dxdt0, y0 = odefun(time[p + 1], x0, u[:, p + 1], c)

        # save states (solution), state derivitives, and outputs
        x_sol[:, p+1] = x0    # state
        x_drv[:, p+1] = dxdt1 # state derivitives
        y_sol[:, p+1] = y0    # output

        # safety - incase NaN or Inf
#       if not np.all(np.abs(x0) > 1e12):
        if not np.all(np.isfinite(x0)):
            break

    return time, x_sol, x_drv, y_sol

Question 3: The model as provided has one ocean that covers the entire planet.

(a) Revise the code to account for the
 fact that the land area fraction of the land is covered by land, which is simulated by a water depth
 of land thickness meters, (and that the full ocean accounts for (1- land area fraction) of the
 surface.)
 Re-run the simulation. You will see that the temperatures are no longer in equilibrium.

 (b) Re-tune the heat transfer coefficients so that the land / ocean / atmosphere model is in equilibrium
 by iteratively adjusting the atm heat transfer coefficient, the ocn heat transfer coefficient and the
 ocn atm heat transfer coefficient. In the tuned model the temperature at the bottom of the ocean
 should be constant somewhere between 1 deg C and 4 deg C, the temperature in the top layer of the
 ocean should be about 19 deg C, and the temperature in the bottom layer of the atmosphere should
 be about 15 deg C

Answer (a) After running the simulation for 15 years, the system is no longer in equilibrium. The ocean surface temperature increased from 19°C to 20.47°C, and the bottom atmospheric layer warmed from 15°C to 15.93°C. This departure from equilibrium occurs because the original heat transfer coefficients were tuned for 100% ocean coverage. The addition of land (with much lower heat capacity due to shallow depth) disrupts the energy balance, requiring re-tuning of the heat transfer parameters.

Changes for Question 3 were marked with "# Question 3" comments

In [ ]:
#!/usr/bin/env python3

# 2N layer heat energy model for the earth's oceans and atmosphere

# https://en.wikipedia.org/wiki/Ocean_temperature
# https://en.wikipedia.org/wiki/Idealized_greenhouse_model
# https://en.wikipedia.org/wiki/Heat_transfer_coefficient
# https://www.engineersedge.com/thermodynamics/overall_heat_transfer-table.htm
# https://enghandbook.com/thermodynamics/heat-transfer/
# https://www.engineeringtoolbox.com/water-steam-thermal-diffusivity-d_2058.html

''' ... in console ...
import importlib , earth_heat_2N
importlib.reload(earth_heat_2N)     # to load edits
'''

# planetary constants
# -------------------
class c:
    pi  =   np.pi
    c   =  299.792458e6         # speed of light  m/s
    h   =    6.62607015e-34     # Planck's constant  J/Hz
    k   =    1.380649e-23       # Boltzmann's constant J/deg K
    sbc =    5.670367e-8        # Stefan-Boltzman constant  ... W/sq.m/deg K^4
    Ts  = 5777.                 # sun temperature, K
    Rs  =    6.9570e08          # radius of sun, m
    D   =    1.4960e11          # sun - to - earth distance, m
    Re  =    6.3567523e6        # radius of earth, m
    sec_per_yr = 365*24*3600    # seconds per year
    aa     =  0.35            #?# earth albedo 0.3 to 0.4 frctn of solar reflected
    ee_atm =  0.78            #?# atmos emissivity without GHG: 0.78 with GHG: 0.80
    ee_ocn =  0.96            #?# ocean emissivity 0.95 to 0.98
    years  = 20                 # years in the simulation
    n_layer = 8                 # layers in the ocean, layers in the atmosphere
#   RF = 4.00                 #?# radiative forcing imbalance  W/sq.m
#   lambda_LW = 4/3           #?# climate feedback parameter W/(sq.m K)
    T_strat = 273 - 60          # temperature of the stratosphere, K
    atm_density_s = 1.23        # density of air at sea level kg/cu.m
    atm_density_t = 0.32        # density of air at top of troposphere kg/cu.m
    atm_shc = 1003              # specific heat capacity of air J/(kg K)
    atm_heat_transfer =  3.7  #?# heat transfer rate of atm 5-30 W/(sq.m K)
    atm_elvtn = 12650            # depth of troposphere m
#   atm_water_cont= 0.004       # volume fraction of water in troposhere
    ocn_atm_heat_transfer=40  #?# heat transfer rate ocn-atm 5 to 50 W/(sq.m K)
    ocn_shc = 4000              # seawater specific heat capacity J/(kg K)
    ocn_density = 1023.6        # seawater density kg/cu.m

    #FOR QUESTION 3 PART A
    # ocn_heat_transfer =  4.0  #?# heat transfer rate of ocn 1-30 W / (sq.m K)

    #FOR QUESTION 4 PART A - FROM RETUNING OUTPUT FROM QUESTION 3 PART B
    ocn_heat_transfer =  5.5  #?# heat transfer rate of ocn 1-30 W / (sq.m K)

    ocn_depth = 3150            # average ocean depth m
    land_thickness = 8.4        # equivalent water thickness of land, m
    land_area_fraction = 0.292  # fraction of earth that is land . unitless
    land_depth = 8.4
    ocean_area_fraction = 1 - 0.292

    # NEW: Calculate land layer properties FOR QUESTION 3
    land_z = land_depth * np.hstack([0, -np.logspace(-1.5, 0.0, n_layer)])
    land_tz = land_z[0:n_layer] - land_z[1:n_layer+1]
    land_ahc = ocn_shc * ocn_density * land_tz  # land acts like shallow water

    # computed constants
    # ------------------
    solar_const = sbc*Ts**4.0 * 4*pi*Rs**2 / (4*pi*D**2.0) # solar constant W/sq.m

    # elevation of layers in the atm
    atm_z = atm_elvtn * np.hstack( [ 0 ,  np.logspace(-1.5,0.0,n_layer) ] )
    # elevation of layers in the ocn
    ocn_z = ocn_depth * np.hstack( [ 0 , -np.logspace(-1.5,0.0,n_layer) ] )

    atm_tz = atm_z[1:n_layer+1] - atm_z[0:n_layer]   # thickness of atm layer
    ocn_tz = ocn_z[0:n_layer]   - ocn_z[1:n_layer+1] # thickness of ocn layer

    atm_dz = 0.5*(atm_z[2:n_layer+1] - atm_z[0:n_layer-1]) # distance between atm layers
    ocn_dz = 0.5*(ocn_z[0:n_layer-1] - ocn_z[2:n_layer+1]) # distance between ocn layers
    ao__dz = 0.5*(atm_z[1] - ocn_z[1])                 # distance between atm&ocn layers

#   print("atm_z = " , np.round(atm_z))
#   print("ocn_z = " , np.round(ocn_z))
    print("ao__dz = ", np.round(ao__dz))

#   atmospheric density linear with altitude
#   atm_density = atm_density_s + (atm_density_t-atm_density_s)*atm_z/atm_elvtn
#   atmospheric density exponential with altitude
    atm_density_decay = atm_elvtn/np.log(atm_density_s / atm_density_t)
    atm_density = atm_density_s * np.exp(-(atm_z[0:n_layer]+0.5*atm_tz)/atm_density_decay)

#   print("atm_density", atm_density)

#   troposphere area heat capacity (per sq.m) J/(sq.m K)
    atm_ahc = atm_shc * atm_density * atm_tz
#   atm_ahc = ocn_shc * ocn_density * 50.0 # 50 m of water in 13000 m of atm
#   ocean       area heat capacity (per sq.m) J/(sq.m K)
    ocn_ahc = ocn_shc * ocn_density * ocn_tz


# FOR QUESTION 4
    # Spatially-varying heat transfer coefficients (exponential profiles)
    # -------------------------------------------------------------------

    # Atmosphere: heat transfer decreases with altitude (like density)
    # At sea level: atm_heat_transfer, at top: lower value
    atm_ht_surface = 3.7              # surface value W/(m²·K)
    atm_ht_top = 0.5                  # top of troposphere value W/(m²·K)
    atm_ht_decay = atm_elvtn / np.log(atm_ht_surface / atm_ht_top)
    atm_ht = atm_ht_surface * np.exp(-(atm_z[0:n_layer] + 0.5*atm_tz) / atm_ht_decay)

    # Ocean: heat transfer increases with depth (mixing is stronger in deep ocean)
    # At surface: ocn_heat_transfer, at bottom: higher value
    ocn_ht_surface = 5.5              # surface value W/(m²·K)
    ocn_ht_bottom = 15.0              # bottom value W/(m²·K)
    ocn_ht_decay = -ocn_depth / np.log(ocn_ht_bottom / ocn_ht_surface)
    # Note: negative depth, so use absolute value of ocn_z
    ocn_ht = ocn_ht_surface * np.exp(-(-ocn_z[0:n_layer] - 0.5*ocn_tz) / ocn_ht_decay)

    # Land: similar to ocean but scaled to shallow depth
    land_ht_surface = 5.5
    land_ht_bottom = 8.0
    land_ht_decay = -land_thickness / np.log(land_ht_bottom / land_ht_surface)
    land_ht = land_ht_surface * np.exp(-(-land_z[0:n_layer] - 0.5*land_tz) / land_ht_decay)

    print("atm_ht (layers) = ", np.round(atm_ht, 2))
    print("ocn_ht (layers) = ", np.round(ocn_ht, 2))
    print("land_ht (layers) = ", np.round(land_ht, 2))

In [ ]:
# define the dynamical system in terms of a set of o.d.e's
# --------------------------------------------------------
def sys(t, x, u, c):
    """
    compute the state derivives dxdt given current state, time, and input
    x = [ Q_atm , Q_ocn ] states ... heat in atmosphere and ocean J/sq.m
    """

    N  = c.n_layer
    # extract individual states from the vector of states , J/sq.m
    Q_atm = x[ 0 :   N ]       #   x( 0 :   N-1 )
    Q_ocn = x[ N : 2*N ]       #   x( N : 2*N-1 )
    Q_land = x[2*N:3*N]  # NEW FOR QUESTION 3

#   print("x= ", x)
#   print("Q_atm= ", Q_atm)
#   print("Q_ocn= ", Q_ocn)

    # convert from heat to temperature , K
    T_atm = np.maximum( 0.0 , Q_atm / c.atm_ahc )    # atmosphere deg K
    T_ocn = np.maximum( 0.0 , Q_ocn / c.ocn_ahc )    #   ocean    deg K

    # NEW FOR QUESTION 3
    T_land = np.maximum( 0.0 , Q_land / c.land_ahc )  # land deg K

#   print("T_atm = ", T_atm)
#   print("T_ocn = ", T_ocn)

#   Atmosphere Heat .....................................
    dQ_atm_dt = np.ones(N)*np.nan

    R_out_U = c.ee_ocn * c.sbc * T_ocn[0]**4.0

    # FOR QUESTION 3
    h_U = c.atm_heat_transfer

    for k in range(N):
        # FOR QUESTION 3
        # T_L = T_atm[k-1]
        # h_L = c.atm_heat_transfer
        # if k == 0:
        #     T_L = T_ocn[0]
        #     h_L = c.ocn_atm_heat_transfer
        # if k == N-1:
        #     T_U = c.T_strat
        # else:
        #     T_U = T_atm[k+1]

      # FOR QUESTION 4

        T_L = T_atm[k-1]
        h_L = c.atm_ht[k-1] if k > 0 else c.atm_ht[0]  # <-- Use array
        if k == 0:
            T_L = c.ocean_area_fraction * T_ocn[0] + c.land_area_fraction * T_land[0]
            h_L = c.ocn_atm_heat_transfer  # Keep this constant (it's the interface)
        if k == N-1:
            T_U = c.T_strat
        else:
            T_U = T_atm[k+1]

        # Heat transfer to upper layer
        if k == N-1:
            h_U = c.atm_ht[k]              # <-- Use array
        else:
            h_U = 0.5 * (c.atm_ht[k] + c.atm_ht[k+1])

        # radiative inputs and outputs in an atmosphere layer
        # ---------------------------------------------------
        # radiant heat input through a layer's lower surface is
        # the radiant heat out of the lower layer's upper surface ...
        R_in_L  = R_out_U
        # radiant heat input through a layer's upper surface is
        # the radiant heat out of the upper layer's lower surface ...
        R_in_U  = c.ee_atm * c.sbc * T_U**4.0
        # radiant heat output through a layer's lower surface
        # depends on the layer's emissivity and the layer's temperature
        # and the radiant heat from transmitted through
        # the layer's upper surface that is not absorbed by the layer
        R_out_L = (1-c.ee_atm) * R_in_U  +  c.ee_atm * c.sbc * T_atm[k]**4.0
        # radiant heat output through a layer's upper surface
        # depends on the layer's emissivity and the layer's temperature
        # and the radiant heat from transmitted through
        # the layer's upper surface that is not absorbed by the layer
        R_out_U = (1-c.ee_atm) * R_in_L  +  c.ee_atm * c.sbc * T_atm[k]**4.0

        # rate of change in heat within atmosphere layer number "k"
        dQ_atm_dt[k] = R_in_L + R_in_U - R_out_L - R_out_U \
                       + h_L * (T_L - T_atm[k])  +  h_U * (T_U - T_atm[k])

#   Ocean Heat ......................................
    dQ_ocn_dt = np.ones(N)*np.nan

    h_L = c.ocn_heat_transfer

    for k in range(N):
       # FOR QUESTION 3
        # T_U = T_ocn[k-1]
        # h_U = c.ocn_heat_transfer
        # u0 = 0
        # if k == 0:
        #     T_U = T_atm[0]
        #     h_U = c.ocn_atm_heat_transfer
        #     u0 = u[0]  -  c.ee_ocn*c.sbc*T_ocn[0]**4.0 \
        #         + (1-c.ee_atm)*c.sbc*T_atm[1]**4.0 \
        #         +     c.ee_atm*c.sbc*T_atm[0]**4.0
        # if k == N-1:
        #     h_L = 0
        #     T_L = T_ocn[k]
        # else:
        #     T_L = T_ocn[k+1]

      # FOR QUESTION 4

        T_U = T_ocn[k-1]
        h_U = 0.5 * (c.ocn_ht[k-1] + c.ocn_ht[k]) if k > 0 else c.ocn_ht[0]  # <-- Use array
        u0 = 0
        if k == 0:
            T_U = T_atm[0]
            h_U = c.ocn_atm_heat_transfer
            u0 = c.ocean_area_fraction * u[0] - c.ee_ocn*c.sbc*T_ocn[0]**4.0 \
                + (1-c.ee_atm)*c.sbc*T_atm[1]**4.0 \
                + c.ee_atm*c.sbc*T_atm[0]**4.0
        if k == N-1:
            h_L = 0
            T_L = T_ocn[k]
        else:
            T_L = T_ocn[k+1]
            h_L = 0.5 * (c.ocn_ht[k] + c.ocn_ht[k+1])  # <-- Use array

        # rate of change in heat within ocean layer number "k"
        dQ_ocn_dt[k] = u0 + h_L * (T_L - T_ocn[k]) + h_U * (T_U - T_ocn[k])

#   Land Heat (similar to ocean but with land properties).....................
    dQ_land_dt = np.ones(N)*np.nan

    for k in range(N):
        T_U = T_land[k-1]
        h_U = c.ocn_heat_transfer  # reuse ocean heat transfer
        u0 = 0

        if k == 0:
            T_U = T_atm[0]
            # h_U = c.ocn_atm_heat_transfer

            h_U = 0.5 * (c.land_ht[k-1] + c.land_ht[k]) if k > 0 else c.land_ht[0]  # <-- CHANGED FOR QUESTION 4
            # Solar input and radiation (same as ocean)
            u0 = u[0] - c.ee_ocn*c.sbc*T_land[0]**4.0 \
                + (1-c.ee_atm)*c.sbc*T_atm[1]**4.0 \
                + c.ee_atm*c.sbc*T_atm[0]**4.0

        if k == N-1:
            h_L = 0
            T_L = T_land[k]
        else:
            T_L = T_land[k+1]
            h_L = 0.5 * (c.land_ht[k] + c.land_ht[k+1])  # <-- CHANGED FOR QUESTION 4

        dQ_land_dt[k] = u0 + h_L * (T_L - T_land[k]) + h_U * (T_U - T_land[k])

#   RF Radiative Feedback ...........................
    RF = c.sbc * ( (1-c.ee_atm)*T_atm[1]**4.0 + c.ee_atm * T_atm[0]**4.0 \
                                              - c.ee_ocn * T_ocn[0]**4.0 )
#   print("RF = ", RF)

#   new state derivative ......
    dxdt = np.hstack([dQ_atm_dt, dQ_ocn_dt, dQ_land_dt])

#   new output ...........
    y = np.hstack([T_atm, T_ocn, T_land, RF])

#   state derivitive ................................
    # dxdt = np.hstack( [ dQ_atm_dt , dQ_ocn_dt ] ) # rate of change of heat per sq.m

#   output ..........................................
    # y = np.hstack( [ T_atm , T_ocn , RF  ] )



    return dxdt, y

In [ ]:
# simulate the dynamical system
# -----------------------------
def run():
    N = c.n_layer
    # time
    # --------------------
    time_end     = c.sec_per_yr  * c.years           # final time, s
    time_step    = c.sec_per_yr  / (12*365)          # time step, s
    n_time       = int( time_end / time_step )
    dt           = time_step * np.ones(n_time)
    n_ramp       = int( c.sec_per_yr * 9 / time_step )
    k_ramp       = np.linspace(0,n_ramp-1,n_ramp)
    dt[0:n_ramp] = time_step*( np.sin(2*np.pi*k_ramp/n_ramp/4) )**2.0
    time         = np.cumsum(dt);

    print("time_step=", time_step/3600, "hr ...  number of time steps=", n_time)
#   print("n_time_init=", n_time_init, "time_step_init=", time_step_init )

    # initial state values
    # --------------------
    # initial temp in the atmosphere and oceans deg.K
#   T_atm_init = np.linspace( 1 , -1 , N )
#   T_ocn_init = np.linspace( 1 ,  1 , N )
#   T_atm_init = np.linspace( 15 , -59 , N )
#   T_ocn_init = np.linspace( 17 ,   2 , N )
    T_atm_init = np.array( [ 15.,  -3.,  -17., -30., -40., -49., -55., -59.] )
    T_ocn_init = np.array( [ 19.,  17.,   14.,  12.,   9.,   7.,   4.,   3.] )

    T_land_init = np.array([19., 17., 14., 12., 9., 7., 4., 3.])  # same as ocean FOR QUESTION 3

    print("T_atm_init = ", np.round( T_atm_init ) )
    print("T_ocn_init = ", np.round( T_ocn_init ) )

    x_init = np.hstack([ c.atm_ahc * (T_atm_init + 273),
                         c.ocn_ahc * (T_ocn_init + 273),
                         c.land_ahc * (T_land_init + 273) ])  # NEW FOR QUESTION 3

    # x_init = np.hstack([ c.atm_ahc*(T_atm_init+273) , \
    #                      c.ocn_ahc*(T_ocn_init+273) ])

#   print("x_init = ", x_init)

    # external forcing
    # --------------------
    # short wavelength solar input over the surface W/sq.m
    u = (1-c.aa)*(c.solar_const/4)*np.ones(n_time)

#   print("u0 = ", u[0])

    # solve the ode's for heat and temperature in the atmosphere and ocean
    # --------------------------------------------------------------------
    time, x, x_drv, y = ode4u( sys , time , x_init , u , c )

    time = time / c.sec_per_yr

    T_atm = y[0:N,:]   - 273;    # deg C
    T_ocn = y[N:2*N,:] - 273;    # deg C

    T_land = y[2*N:3*N, :] - 273  # NEW FOR QUESTION 3

    RF    = y[2*N];

    # print some results
    # ----------------------------------------------------
#   print("sbc=",c.sbc, "So=", c.So, "u=",u[0])
    print("Year= ",np.round(time[-1]))
    print("The temperatures in the atmosphere layers:" )
    print(" T_atm",np.round(time[-1]),"= ",np.round(T_atm[:,-1]))
    print(" T_atm[0]",np.round(time[-1]),"= ",T_atm[0,-1])
    print("The temperatures in the    ocean    layers:" )
    print(" T_ocn",np.round(time[-1]),"= ",np.round(T_ocn[:,-1]))
    print(" T_ocn[0]",np.round(time[-1]),"= ",T_ocn[0,-1])
#   print("atm_ahc=", c.atm_ahc/1e6, "ocn_ahc=", c.ocn_ahc/1e6)

    print("The temperatures in the    land     layers:" )
    print(" T_land",np.round(time[-1]),"= ",np.round(T_land[:,-1]))
    print(" T_land[0]",np.round(time[-1]),"= ",T_land[0,-1])

    # ---------------------------------------------------------- Plots
    fontsize = 12
    plt.rcParams['font.size'] = fontsize
    plt.ion()                 # interactive mode

#   f10 = plt.figure(10) # ---------------------------------------
#   plt.clf()
#   plt.plot(time, dt,  "-", color='Black')
#   plt.ylabel('delta-t')
#   plt.xlabel('Time (years)')
#
#   f11 = plt.figure(11) # ---------------------------------------
#   plt.clf()
#   plt.plot(time,  "-", color='Black')
#   plt.ylabel('t')
#   plt.xlabel('time index number ')
#
#   f0 = plt.figure(0) # ---------------------------------------
#   plt.clf()
#   plt.plot(time, u, "-", color='Red')
#   plt.ylabel('Short Wavelentgh Solar Input at Surface W/sq.m')
#   plt.xlabel('Time (years)')

    f1 = plt.figure(1) # ---------------------------------------
#   fig=plt.figure(figsize=(4.2,4.3),dpi=300)
    plt.clf()
#   plt.subplot(3,1,1)

    plt.plot(time, T_atm[0,:], "-", color='SkyBlue', label="atmosphere")
    plt.plot(time, T_ocn[0,:], "-", color='MidnightBlue', label="ocean")

    plt.plot(time, T_land[0,:], "-", color='SaddleBrown', label="land") # NEW FOR QUESTION 3

    plt.plot(time[-1], T_atm[0,-1], "o", color='SkyBlue')
    plt.plot(time[-1], T_ocn[0,-1], "o", color='MidnightBlue')
    for k in range(1,N):
        plt.plot(time, T_atm[k,:], "--", color='SkyBlue' )
        plt.plot(time, T_ocn[k,:], "--", color='MidnightBlue')
    plt.ylabel('Temperature (°C)')
    plt.xlabel('Time (years)')
#   plt.xlim([0,200])
#   plt.ylim([0,4])
#   plt.title('Atm/ Ocean temperature anomalies given a sudden 2x CO฀ increase')
    plt.legend(loc='lower right',fontsize=fontsize-1)
    plt.grid(lw=0.25)
    plt.title("(a)",loc="left",fontsize=fontsize)

    f2 = plt.figure(2) # ---------------------------------------
#   fig=plt.figure(figsize=(4.2,4.3),dpi=300)
    plt.clf()
#   plt.subplot(3,1,1)

    plt.plot(time, y[2,:], "-", color='Orange')
    plt.ylabel('RF W/sq.m')
    plt.xlabel('Time (years)')
#   plt.xlim([0,200])
#   plt.ylim([0,4])
#   plt.title('Atm/ Ocean temperature anomalies given a sudden 2x CO฀ increase')
#   plt.legend(loc='right',fontsize=fontsize-1)
    plt.grid(lw=0.25)
    plt.title("(b)",loc="left",fontsize=fontsize)

    plt.show()

In [ ]:
run()

## For QUES 3 Part B

The addition of land (with much lower thermal mass due to its shallow 8.4m depth) disrupted the original energy balance. The key to restoring equilibrium was increasing vertical heat transfer within the ocean (ocn_heat_transfer), which more efficiently transports heat from the warm surface layers to cooler deep layers. This prevents excessive heat accumulation at the surface while maintaining realistic deep ocean temperatures. The land-ocean split requires stronger vertical ocean mixing to compensate for the reduced overall heat capacity of the surface.

In [ ]:
# Retuning Try 1
print("\n" + "="*60)
print("TESTING NEW PARAMETERS")
print("="*60)

# Modify parameters
c.atm_heat_transfer = 3.7
c.ocn_heat_transfer = 4.0
c.ocn_atm_heat_transfer = 36  # Changed from 40

# Run again
run()

In [ ]:
# Retuning Try 2
print("\n" + "="*60)
print("TEST 2: INCREASED COUPLING")
print("="*60)

c.atm_heat_transfer = 3.7
c.ocn_heat_transfer = 4.0
c.ocn_atm_heat_transfer = 44  # Increase from 40 to 44

run()

In [ ]:
# Retuning try 3
print("\n" + "="*60)
print("TEST 3: INCREASED OCEAN MIXING")
print("="*60)

c.atm_heat_transfer = 3.7
c.ocn_heat_transfer = 5.5  # Increase from 4.0 to 5.5
c.ocn_atm_heat_transfer = 40  # Back to original

run()

## Ques 4

(A) Within the atmosphere and in the oceans the heat transfer coefficient varies with height. Ask LLM
 how the ocean heat transfer coefficent and the the atmosphere what the heat transfer coefficients are at
 sea level and the top of the atmosphere and the bottom of the ocean.

 (B)Ask LLM to revise the sys
 function and the sim function to account for spatially varying heat transfer coefficients. by modeling
 these sptially-varying heat transfer coefficients with an exponential, as the atmospheric density is
 modeled.

 (C)Retune the model by again adjusting the three heat transfer coefficents at the surface so that the land
 / sea / atmosphere model is in equilibrium. with ocean surface temperatures at around 19 deg C and
 atmosphere surface temperature at around 15 deg C.
 What are the surface level heat transfer coefficients and the top-of-troposphere and bottom-of-ocean
 heat transfer coefficients?

CODE CHANGES: Changes were marked with comments "# Question 4" in the above code cells

Answer PART (A) Claude's Response:

Heat transfer coefficients vary significantly with altitude in the atmosphere and depth in the ocean due to changes in fluid density, turbulence, and mixing processes.

Atmosphere: The atmospheric heat transfer coefficient decreases with altitude, primarily because air density decreases exponentially with height. At sea level, convective heat transfer is strong due to higher density and turbulent mixing near the surface.

1. At sea level: Typical values range from 5-25 W/(m²·K) depending on wind speed and surface roughness. For large-scale climate modeling, values around 3-5 W/(m²·K) are common for vertical heat diffusion.
2. At top of troposphere (~12 km): Much lower, approximately 0.5-2 W/(m²·K), as the air is thinner and mixing is weaker.

The variation roughly follows an exponential decay similar to atmospheric density: h(z) ≈ h₀ × exp(-z/H), where H is a scale height.

Ocean: Ocean heat transfer coefficients (related to vertical eddy diffusivity) show the opposite trend - they generally increase with depth, though the pattern is complex.

1. Near surface: Relatively low, around 1-10 W/(m²·K) in the upper mixed layer, due to stable stratification that inhibits vertical mixing.
2. At depth (~3000m): Higher values, typically 10-30 W/(m²·K) in the deep ocean, due to internal wave breaking, bottom roughness, and deep circulation patterns.

The ocean's vertical mixing is weaker overall compared to the atmosphere because water is much denser and stratification is stronger. However, deep ocean mixing is enhanced by processes like internal tides and geothermal heating near the seafloor.

Ocean-Atmosphere Interface:
At the surface boundary between ocean and atmosphere, heat transfer coefficients are much higher (30-100 W/(m²·K)) due to turbulent air-sea interaction, wave action, and wind stress.


Answer PART (B) The dramatic temperature increases indicate that the spatially-varying coefficients have fundamentally changed the system's energy balance. Heat is accumulating at the surface because:

1. The reduced atmospheric mixing at altitude (atm_ht drops from 3.58 to 0.74) creates a "lid" that traps heat
2. The relatively small increase in ocean mixing (ocn_ht only ranges 5.41 to 8.35) is insufficient to transport excess surface heat downward quickly enough
3. The very shallow land (8.4m) with weak mixing heats up extremely rapidly

The system has not reached equilibrium after 15 years - temperatures are still rising, as shown by the upward-sloping curves in the plot. Re-tuning of the surface-level heat transfer coefficients (Part C) is necessary to restore energy balance.


## FOR PART C

The tuned surface values differ significantly from Question 3's constant coefficients (atm=3.7, ocn=5.5). With spatially-varying profiles, the surface ocean coefficient needed to increase from 5.5 to 12.0 W/(m²·K) to compensate for the weaker mixing at altitude (atmosphere drops to 0.74 at top) that creates a "thermal lid" trapping heat. The exponential profiles produce more realistic physics: atmospheric mixing decreases with altitude following density reduction, while ocean mixing increases moderately with depth reflecting stronger turbulence in the deep ocean

In [ ]:
#Retuning Try 1
print("\n" + "="*60)
print("TEST 1: INCREASED OCEAN SURFACE MIXING")
print("="*60)

# Modify surface-level parameters
c.atm_ht_surface = 3.7
c.ocn_ht_surface = 12.0  # Increase from 5.5 to 12.0
c.ocn_atm_heat_transfer = 40

# Recalculate the profiles with new surface values
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

print("New atm_ht = ", np.round(c.atm_ht, 2))
print("New ocn_ht = ", np.round(c.ocn_ht, 2))

run()

In [ ]:
# Retuning Try 2
print("\n" + "="*60)
print("TEST 2: MUCH STRONGER MIXING")
print("="*60)

# Increase both ocean AND atmosphere surface mixing
c.atm_ht_surface = 8.0   # Increase from 3.7 to 8.0
c.ocn_ht_surface = 25.0  # Increase from 12.0 to 25.0
c.ocn_atm_heat_transfer = 40

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface  # Match ocean
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

print("New atm_ht = ", np.round(c.atm_ht, 2))
print("New ocn_ht = ", np.round(c.ocn_ht, 2))

run()

In [ ]:
#Retuning Try 3
print("\n" + "="*60)
print("TEST 3: MODERATE MIXING")
print("="*60)

# Split the difference
c.atm_ht_surface = 5.5   # Between 3.7 and 8.0
c.ocn_ht_surface = 18.0  # Between 12.0 and 25.0
c.ocn_atm_heat_transfer = 40

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

print("New atm_ht = ", np.round(c.atm_ht, 2))
print("New ocn_ht = ", np.round(c.ocn_ht, 2))

run()

In [ ]:
#Retuning Try 4

print("\n" + "="*60)
print("TEST 4: FINE-TUNING")
print("="*60)

# Values between Test 1 (too hot) and Test 3 (too cold)
c.atm_ht_surface = 4.5   # Between 3.7 and 5.5
c.ocn_ht_surface = 15.0  # Between 12.0 and 18.0
c.ocn_atm_heat_transfer = 40

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

print("New atm_ht = ", np.round(c.atm_ht, 2))
print("New ocn_ht = ", np.round(c.ocn_ht, 2))

run()

In [ ]:
#Retuning Try 5

print("\n" + "="*60)
print("TEST 5: FURTHER FINE-TUNING")
print("="*60)

# Reduce mixing even more (closer to Test 1)
c.atm_ht_surface = 4.0   # Between 3.7 and 4.5
c.ocn_ht_surface = 13.5  # Between 12.0 and 15.0
c.ocn_atm_heat_transfer = 40

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

print("New atm_ht = ", np.round(c.atm_ht, 2))
print("New ocn_ht = ", np.round(c.ocn_ht, 2))

run()

In [ ]:
#Retuning Try 6

print("\n" + "="*60)
print("TEST 6: VERY FINE ADJUSTMENT")
print("="*60)

# Small adjustments
c.atm_ht_surface = 3.8   # Between 3.7 and 4.0
c.ocn_ht_surface = 12.5  # Between 12.0 and 13.5
c.ocn_atm_heat_transfer = 40

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

print("New atm_ht = ", np.round(c.atm_ht, 2))
print("New ocn_ht = ", np.round(c.ocn_ht, 2))

run()

In [ ]:
#Retuning Try 7

print("\n" + "="*60)
print("TEST 7: FINAL TUNING")
print("="*60)

# Very small adjustment to get the last few degrees
c.atm_ht_surface = 3.75  # Between 3.7 and 3.8
c.ocn_ht_surface = 12.0  # Back towards 12.0
c.ocn_atm_heat_transfer = 40

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

print("New atm_ht = ", np.round(c.atm_ht, 2))
print("New ocn_ht = ", np.round(c.ocn_ht, 2))

run()

## Question 5

 Finally assess the sensitivity to changes in heat transfer coefficients by changing the time span of the
 simulation years to 100 years. and individually changing the three heat transfer coefficients by +1
 and-1 W/sq.m
 What are the effects of these changes upon the surface temperatures in the (land/ocean) and atmo
sphere?

# Baseline Configuration (100 years):

atm_ht_surface = 3.75 W/(m²·K)
ocn_ht_surface = 12.0 W/(m²·K)
ocn_atm_heat_transfer = 40 W/(m²·K)
Results: T_atm[0] = 24.86°C, T_ocn[0] = 27.43°C, T_land[0] ≈ 36.5°C

# Sensitivity Test Results:
1. Atmosphere Surface Mixing (atm_ht_surface):

+1 W/(m²·K): ΔT_atm = -1.80°C, ΔT_ocn = -1.72°C, ΔT_land = -1.8°C
-1 W/(m²·K): ΔT_atm = +2.08°C, ΔT_ocn = +1.97°C, ΔT_land = +1.7°C

2. Ocean Surface Mixing (ocn_ht_surface):

+1 W/(m²·K): ΔT_atm = -3.47°C, ΔT_ocn = -3.40°C, ΔT_land = -4.6°C
-1 W/(m²·K): ΔT_atm = +3.23°C, ΔT_ocn = +3.17°C, ΔT_land = +3.9°C

3. Ocean-Atmosphere Interface (ocn_atm_heat_transfer):

+1 W/(m²·K): ΔT_atm = +0.73°C, ΔT_ocn = +0.65°C, ΔT_land = +0.5°C
-1 W/(m²·K): ΔT_atm = -0.74°C, ΔT_ocn = -0.66°C, ΔT_land = -0.9°C



# Sensitivity Ranking:

1. Ocean surface mixing (ocn_ht_surface): Highest sensitivity (±3.2-3.5°C)
2. Atmosphere surface mixing (atm_ht_surface): Moderate sensitivity (±1.8-2.1°C)
3. Ocean-atmosphere interface (ocn_atm_heat_transfer): Lowest sensitivity (±0.65-0.74°C)

Effects on Surface Temperatures:

1. Ocean Surface Mixing (most sensitive):

Increasing by +1 W/(m²·K): Cooled atmosphere by 3.5°C, ocean by 3.4°C, land by 4.6°C
Decreasing by -1 W/(m²·K): Warmed atmosphere by 3.2°C, ocean by 3.2°C, land by 3.9°C

Physical mechanism: Controls vertical heat transport in ocean. Higher values transport more surface heat to deep layers, cooling the surface. Lower values trap heat near surface, causing warming.

2. Atmosphere Surface Mixing (moderate sensitivity):

Increasing by +1 W/(m²·K): Cooled atmosphere by 1.8°C, ocean by 1.7°C
Decreasing by -1 W/(m²·K): Warmed atmosphere by 2.1°C, ocean by 2.0°C

Physical mechanism: Stronger vertical mixing in atmosphere distributes heat more uniformly, reducing surface accumulation. Weaker mixing creates a thermal "lid" that traps heat near surface.

3. Ocean-Atmosphere Interface (least sensitive):

Increasing by +1 W/(m²·K): Minimal warming (~0.7°C)

Decreasing by -1 W/(m²·K): Minimal cooling (~0.7°C)

Physical mechanism: Interface is already strongly coupled (h=40), so ±1 perturbations have limited effect on heat exchange between surface and atmosphere.

# Key Insights:
The ocean surface mixing coefficient is by far the most important parameter, with approximately 3-5× greater impact than atmosphere mixing and 5× greater impact than the interface coupling. This reflects the ocean's dominant role as Earth's primary heat reservoir. The ocean's large heat capacity means that even small changes in vertical mixing rates dramatically affect how much solar energy accumulates at the surface versus being stored in deeper layers. Land temperatures show the largest absolute changes because the shallow equivalent water depth (8.4m) provides minimal thermal inertia compared to the deep ocean (3150m).
Long-term Behavior:
All simulations showed continued warming over 100 years, indicating that the coefficients tuned for 15-year equilibrium are not stable over centennial timescales. This suggests the model would require different parameter values for long-term climate projections.

In [ ]:
print("\n" + "="*70)
print("BASELINE: 100 YEARS WITH TUNED COEFFICIENTS")
print("="*70)

# Keep tuned values
c.atm_ht_surface = 3.75
c.ocn_ht_surface = 12.0
c.ocn_atm_heat_transfer = 40
c.years = 100

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

run()

In [ ]:
# TEST A+: atm_ht_surface = 4.75 (+1)

print("\n" + "="*70)
print("TEST A+: atm_ht_surface = 4.75 (+1)")
print("="*70)

c.atm_ht_surface = 4.75  # +1 from 3.75
c.ocn_ht_surface = 12.0
c.ocn_atm_heat_transfer = 40
c.years = 100

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

run()

In [ ]:
#  TEST A-: atm_ht_surface = 2.75 (-1)

print("\n" + "="*70)
print("TEST A-: atm_ht_surface = 2.75 (-1)")
print("="*70)

c.atm_ht_surface = 2.75  # -1 from 3.75
c.ocn_ht_surface = 12.0
c.ocn_atm_heat_transfer = 40
c.years = 100

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

run()

In [ ]:
# TEST B+: ocn_ht_surface = 13.0 (+1)

print("\n" + "="*70)
print("TEST B+: ocn_ht_surface = 13.0 (+1)")
print("="*70)

c.atm_ht_surface = 3.75
c.ocn_ht_surface = 13.0  # +1 from 12.0
c.ocn_atm_heat_transfer = 40
c.years = 100

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

run()

In [ ]:
#TEST B-: ocn_ht_surface = 11.0 (-1)

print("\n" + "="*70)
print("TEST B-: ocn_ht_surface = 11.0 (-1)")
print("="*70)

c.atm_ht_surface = 3.75
c.ocn_ht_surface = 11.0  # -1 from 12.0
c.ocn_atm_heat_transfer = 40
c.years = 100

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

run()

In [ ]:
# TEST C+: ocn_atm_heat_transfer = 41 (+1)

print("\n" + "="*70)
print("TEST C+: ocn_atm_heat_transfer = 41 (+1)")
print("="*70)

c.atm_ht_surface = 3.75
c.ocn_ht_surface = 12.0
c.ocn_atm_heat_transfer = 41  # +1 from 40
c.years = 100

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

run()

In [ ]:
# TEST C-: ocn_atm_heat_transfer = 39 (-1)

print("\n" + "="*70)
print("TEST C-: ocn_atm_heat_transfer = 39 (-1)")
print("="*70)

c.atm_ht_surface = 3.75
c.ocn_ht_surface = 12.0
c.ocn_atm_heat_transfer = 39  # -1 from 40
c.years = 100

# Recalculate profiles
c.atm_ht_decay = c.atm_elvtn / np.log(c.atm_ht_surface / c.atm_ht_top)
c.atm_ht = c.atm_ht_surface * np.exp(-(c.atm_z[0:c.n_layer] + 0.5*c.atm_tz) / c.atm_ht_decay)

c.ocn_ht_decay = -c.ocn_depth / np.log(c.ocn_ht_bottom / c.ocn_ht_surface)
c.ocn_ht = c.ocn_ht_surface * np.exp(-(-c.ocn_z[0:c.n_layer] - 0.5*c.ocn_tz) / c.ocn_ht_decay)

c.land_ht_surface = c.ocn_ht_surface
c.land_ht_decay = -c.land_thickness / np.log(c.land_ht_bottom / c.land_ht_surface)
c.land_ht = c.land_ht_surface * np.exp(-(-c.land_z[0:c.n_layer] - 0.5*c.land_tz) / c.land_ht_decay)

run()